# 7. 인물사진을 만들어 보자

## 7-8. 프로젝트: 인물 모드 문제점 찾기

---

## 목차

### Step 1. 인물모드 직접 해 보기

### Step 2. 사진에서 문제점 찾기

### Step 3. 해결 방법을 제안해 보기

---

- 라이브러리 불러오기

In [ ]:
import os
import urllib
import cv2
import numpy as np
from pixellib.semantic import semantic_segmentation
from matplotlib import pyplot as plt

print('슝=3')

- DeepLab 모델 불러온 후 생성

In [ ]:
MODEL_DIR = 'human_segmentation/models'
MODEL_FILE = os.path.join(MODEL_DIR, 'deeplabv3_xception_tf_dim_ordering_tf_kernels.h5')

if not os.path.exists(MODEL_FILE):
    # PixelLib가 제공하는 모델의 url입니다
    MODEL_URL = 'https://github.com/ayoolaolafenwa/PixelLib/releases/download/1.1/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5'

    # 다운로드를 시작합니다
    urllib.request.urlretrieve(MODEL_URL, MODEL_FILE)

In [ ]:
# 세그멘테이션 모델 생성
MODEL = semantic_segmentation()
MODEL.load_pascalvoc_model(MODEL_FILE)

- PASCAL VOC 데이터 라벨 종류

In [ ]:
LABEL_NAMES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]
len(LABEL_NAMES)

### Step 1. 인물모드 직접 해 보기

- 여러분의 셀카를 이용해서 오늘 배운 내용을 수행해 봅시다. 아래와 같은 이미지를 얻어야 합니다. 최소 3장 이상의 인물모드 사진을 만들어 봅시다.
- 인물이 주인공이 아닌, 귀여운 고양이에 대한 아웃포커싱 사진도 만들어 볼 수 있을 것입니다. 시맨틱 세그멘테이션 스텝에서 힌트를 찾아봅시다.
- 배경을 blur하는 인물모드 사진이 아니라 배경사진을 다른 이미지로 교체하는 크로마키 배경 합성을 시도해 볼 수도 있을 것입니다. 여러분만의 환상적인 사진을 만들어 보면 어떨까요?

In [ ]:
def get_colormap(label_name):
    # 아래 코드를 이해하지 않아도 좋습니다
    # PixelLib에서 그대로 가져온 코드입니다
    # 주목해야 할 것은 생상 코드 결과물이예요!
    colormap = np.zeros((256, 3), dtype = int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
            colormap[:, channel] |= ((ind >> channel) & 1) << shift
        ind >>= 3

    # colormap[:20]

    index = LABEL_NAMES.index(label_name)  # 데이터 라벨 인덱스
    seg_color = colormap[index].tolist()  # BGR
    seg_color = seg_color[::-1]  # BGR2RGB
    return seg_color  # RGB

In [ ]:
def draw_img(img, img_title='', cmap=None):
    print(img.shape)
    plt.figure(figsize=(12,12))
    plt.imshow(img, cmap)
    plt.title(img_title)
    plt.rc('axes', titlesize=20)
    plt.axis('off')
    plt.show()

In [ ]:
def draw_imgs(img_1, img_2, img_1_title='', img_2_title=''):
    plt.figure(figsize=(20,20))

    plt.subplot(211)
    plt.imshow(img_1)
    plt.title(img_1_title)
    plt.rc('axes', titlesize=20)
    plt.axis('off')

    plt.subplot(212)
    plt.imshow(img_2)
    plt.title(img_2_title)
    plt.rc('axes', titlesize=20)
    plt.axis('off')

    plt.show()

#### 1. 인물 사진

- 이미지 확인

In [ ]:
PERSON_IMG_PATH = 'human_segmentation/images/couple.jpg'
PERSON_IMG_ORIG = cv2.imread(PERSON_IMG_PATH)

In [ ]:
draw_img(cv2.cvtColor(PERSON_IMG_ORIG, cv2.COLOR_BGR2RGB), 'PERSON_IMG_ORIG')

- DeepLab 모델에 이미지 입력 후 출력 확인

In [ ]:
# 모델에 이미지 입력
PERSON_SEGVALUES, PERSON_OUTPUT = MODEL.segmentAsPascalvoc(PERSON_IMG_PATH)

In [ ]:
# 영역 검출된 클래스 출력
for class_id in PERSON_SEGVALUES['class_ids']:
    print(LABEL_NAMES[class_id])

In [ ]:
# 모델 출력값 확인
draw_img(PERSON_OUTPUT, 'PERSON_OUTPUT')

- seg_color로만 이루어진 마스크 생성

In [ ]:
# output의 픽셀 별로 색상이 seg_color와 같다면 1(True), 다르다면 0(False)이 됩니다
seg_color = get_colormap('person')
person_seg_map = np.all(PERSON_OUTPUT==seg_color, axis=-1)

draw_img(person_seg_map, cmap='gray')

- 원본 이미지와 겹쳐서 세그멘테이션 적용 확인

In [ ]:
person_img_show = PERSON_IMG_ORIG.copy()

# True과 False인 값을 각각 255과 0으로 바꿔줍니다
person_img_mask = person_seg_map.astype(np.uint8) * 255

# 255와 0을 적당한 색상으로 바꿔봅니다
person_color_mask = cv2.applyColorMap(person_img_mask, cv2.COLORMAP_JET)

# 원본 이미지와 마스트를 적당히 합쳐봅니다
# 0.6과 0.4는 두 이미지를 섞는 비율입니다.
person_img_show = cv2.addWeighted(person_img_show, 0.6, person_color_mask, 0.4, 0.0)

draw_img(cv2.cvtColor(person_img_show, cv2.COLOR_BGR2RGB))

- 이미지 흐리게 만들기
  - cv2.blur(img, blurring_kernel_size)
    - OpenCV의 blur 함수는 평균 블러링(Averaging Blurring)이다.

In [ ]:
# blurring kernel size = (17,17)
person_img_orig_blur = cv2.blur(PERSON_IMG_ORIG, (17,17))

draw_img(cv2.cvtColor(person_img_orig_blur, cv2.COLOR_BGR2RGB), 'person_img_orig_blur')

- 흐려진 이미지에서 세그멘테이션 마스크를 이용해서 배경만 추출
  - 색이 없으면 검게 보이고(0) 색이 있으면 흰색(1)으로 표현됨
    - cv2.bitwise_not(img): 색 반전 => 0은 1로, 1은 0으로 변경
    - cv2.bitwise_and(img_1, img_2): 모두 흰색(1)인 부분만 1로 나타남

-not 연산

|A|not A|
|:---:|:---:|
|1|0|
|0|1|

-and 연산

|A|B|A and B|
|:---:|:---:|:---:|
|0|0|0|
|1|0|0|
|0|1|0|
|1|1|1|

In [ ]:
person_img_mask_color = cv2.cvtColor(person_img_mask, cv2.COLOR_GRAY2BGR)
person_img_bg_mask = cv2.bitwise_not(person_img_mask_color)
person_img_bg_blur = cv2.bitwise_and(person_img_orig_blur, person_img_bg_mask)

draw_img(cv2.cvtColor(person_img_bg_blur, cv2.COLOR_BGR2RGB))

- 배경 이미지와 사람 이미지 합치기
  - numpy.where(condition,[x,y])
    - 입력 조건을 충족하는 배열의 색인을 생성
    - 조건이 True이면 출력에 x의 요소가 포함되고, 그렇지 않으면 출력에 y의 요소가 포함된다.

In [ ]:
person_img_concat = np.where(person_img_mask_color==255, PERSON_IMG_ORIG, person_img_bg_blur)

draw_img(cv2.cvtColor(person_img_concat, cv2.COLOR_BGR2RGB), 'person_img_concat')

- 이미지 처리 결과 저장

In [ ]:
# cf. plt.savefig() 는 plt.show() 직전에 해야 처리가 끝난 이미지 저장이 제대로 된다.
# plt.savefig('image.png', bbox_inches='tight',pad_inches = 0) 로 저장했더니 공백 및 테두리 숨기기가 제대로 안 됨 => plt.imsave('image.png', img)를 사용함
plt.imsave('result_imgs/person_blur.png', cv2.cvtColor(person_img_concat, cv2.COLOR_BGR2RGB))

- blur 적용 전후 비교

In [ ]:
draw_imgs(cv2.cvtColor(PERSON_IMG_ORIG, cv2.COLOR_BGR2RGB), cv2.cvtColor(person_img_concat, cv2.COLOR_BGR2RGB), 'PERSON_IMG_ORIG', 'person_img_concat')

#### 2. 고양이 사진

- 이미지 확인

In [ ]:
CAT_IMG_PATH = 'human_segmentation/images/cat_1280x853.jpg'  
CAT_IMG_ORIG = cv2.imread(CAT_IMG_PATH) 

In [ ]:
draw_img(cv2.cvtColor(CAT_IMG_ORIG, cv2.COLOR_BGR2RGB), 'CAT_IMG_ORIG')

- DeepLab 모델에 이미지 입력 후 출력 확인

In [ ]:
# 모델에 이미지 입력
CAT_SEGVALUES, CAT_OUTPUT = MODEL.segmentAsPascalvoc(CAT_IMG_PATH)

In [ ]:
# 영역 검출된 클래스 출력
for class_id in CAT_SEGVALUES['class_ids']:
    print(LABEL_NAMES[class_id])

In [ ]:
# 모델 출력값 확인
draw_img(CAT_OUTPUT, 'CAT_OUTPUT')

- seg_color로만 이루어진 마스크 생성

In [ ]:
# output의 픽셀 별로 색상이 seg_color와 같다면 1(True), 다르다면 0(False)이 됩니다
seg_color = get_colormap('cat')
cat_seg_map = np.all(CAT_OUTPUT==seg_color, axis=-1)

draw_img(cat_seg_map, cmap='gray')

- 원본 이미지와 겹쳐서 세그멘테이션 적용 확인

In [ ]:
cat_img_show = CAT_IMG_ORIG.copy()

# True과 False인 값을 각각 255과 0으로 바꿔줍니다
cat_img_mask = cat_seg_map.astype(np.uint8) * 255

# 255와 0을 적당한 색상으로 바꿔봅니다
cat_color_mask = cv2.applyColorMap(cat_img_mask, cv2.COLORMAP_JET)

# 원본 이미지와 마스트를 적당히 합쳐봅니다
# 0.6과 0.4는 두 이미지를 섞는 비율입니다.
cat_img_show = cv2.addWeighted(cat_img_show, 0.6, cat_color_mask, 0.4, 0.0)

draw_img(cv2.cvtColor(cat_img_show, cv2.COLOR_BGR2RGB))

- 이미지 흐리게 만들기
  - cv2.blur(img, blurring_kernel_size)
    - OpenCV의 blur 함수는 평균 블러링(Averaging Blurring)이다.

In [ ]:
# blurring kernel size = (17,17)
cat_img_orig_blur = cv2.blur(CAT_IMG_ORIG, (17,17))

draw_img(cv2.cvtColor(cat_img_orig_blur, cv2.COLOR_BGR2RGB), 'cat_img_orig_blur')

- 흐려진 이미지에서 세그멘테이션 마스크를 이용해서 배경만 추출
  - 색이 없으면 검게 보이고(0) 색이 있으면 흰색(1)으로 표현됨
    - cv2.bitwise_not(img): 색 반전 => 0은 1로, 1은 0으로 변경
    - cv2.bitwise_and(img_1, img_2): 모두 흰색(1)인 부분만 1로 나타남

-not 연산

|A|not A|
|:---:|:---:|
|1|0|
|0|1|

-and 연산

|A|B|A and B|
|:---:|:---:|:---:|
|0|0|0|
|1|0|0|
|0|1|0|
|1|1|1|

In [ ]:
cat_img_mask_color = cv2.cvtColor(cat_img_mask, cv2.COLOR_GRAY2BGR)
cat_img_bg_mask = cv2.bitwise_not(cat_img_mask_color)
cat_img_bg_blur = cv2.bitwise_and(cat_img_orig_blur, cat_img_bg_mask)

draw_img(cv2.cvtColor(cat_img_bg_blur, cv2.COLOR_BGR2RGB))

- 배경 이미지와 고양이 이미지 합치기
  - numpy.where(condition,[x,y])
    - 입력 조건을 충족하는 배열의 색인을 생성
    - 조건이 True이면 출력에 x의 요소가 포함되고, 그렇지 않으면 출력에 y의 요소가 포함된다.

In [ ]:
cat_img_concat = np.where(cat_img_mask_color==255, CAT_IMG_ORIG, cat_img_bg_blur)

draw_img(cv2.cvtColor(cat_img_concat, cv2.COLOR_BGR2RGB), 'cat_img_concat')

- 이미지 처리 결과 저장

In [ ]:
# cf. plt.savefig() 는 plt.show() 직전에 해야 처리가 끝난 이미지 저장이 제대로 된다.
# plt.savefig('image.png', bbox_inches='tight',pad_inches = 0) 로는 공백 및 테두리 숨기기가 제대로 안 됐기에 plt.imsave('image.png', img)를 사용함
plt.imsave("result_imgs/cat_blur.png", cv2.cvtColor(cat_img_concat, cv2.COLOR_BGR2RGB))

- blur 적용 전후 비교

In [ ]:
draw_imgs(cv2.cvtColor(CAT_IMG_ORIG, cv2.COLOR_BGR2RGB), cv2.cvtColor(cat_img_concat, cv2.COLOR_BGR2RGB), 'CAT_IMG_ORIG', 'cat_img_concat')

#### 3. 배경전환 크로마키 사진

**나의 요구사항: 배경 이미지에 있는 커튼에 고양이가 매달려 있게 만들 것이다. 매우 귀여울 것이기 때문!**

- 배경 이미지 확인

In [ ]:
BACK_IMG_PATH = 'human_segmentation/images/pforphoto.jpg'  
BACK_IMG_ORIG = cv2.imread(BACK_IMG_PATH)

In [ ]:
draw_img(cv2.cvtColor(BACK_IMG_ORIG, cv2.COLOR_BGR2RGB), 'BACK_IMG_ORIG')

- 고양이 사진 크기 변경
  - cv2.resize(img, (w,h), interpolation)
    - interpolation 종류: cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4, cv2.INTER_AREA

|interpolation|설명|
|:---:|:---:|
|cv2.INTER_NEAREST|최근방 이웃 보간법|
|cv2.INTER_LINEAR|양선형 보간법(2x2 이웃 픽셀 참조)|
|cv2.INTER_CUBIC|3차회선 보간법(4x4 이웃 픽셀 참조)|
|cv2.INTER_LANCZOS4|Lanczos 보간법 (8x8 이웃 픽셀 참조)|
|cv2.INTER_AREA|영상 축소시 효과적|

In [ ]:
# 고양이만 남기고 나머지는 0(검은색) 처리
cat_only_img_concat = np.where(cat_img_mask_color==255, CAT_IMG_ORIG, 0)

# 고양이 resize
h, w, c = cat_only_img_concat.shape
cat_img_resize = cv2.resize(cat_only_img_concat, (w//4, h//4), cv2.INTER_AREA)

In [ ]:
draw_img(cv2.cvtColor(cat_img_resize, cv2.COLOR_BGR2RGB))

- 배경 이미지와 고양이 이미지 합치기
  - numpy.where(condition,[x,y])
    - 입력 조건을 충족하는 배열의 색인을 생성
    - 조건이 True이면 출력에 x의 요소가 포함되고, 그렇지 않으면 출력에 y의 요소가 포함된다.
  - cv2.addWeighted(img_1, a, img_2, b, c)
    - result = img_1 * a + img_2 * b + c
    - b = 1 - a
  - [E-03]에서 사용했던 코드 재활용

In [ ]:
back_img_show = BACK_IMG_ORIG.copy()

refined_y = 200
refined_x = 400

alpha = 0.2  # 투명도
sticker_area = back_img_show[refined_y:refined_y+cat_img_resize.shape[0], refined_x:refined_x+cat_img_resize.shape[1]]
back_img_show[refined_y:refined_y +cat_img_resize.shape[0], refined_x:refined_x+cat_img_resize.shape[1]] = \
cv2.addWeighted(sticker_area, alpha, np.where(cat_img_resize==0,sticker_area,cat_img_resize).astype(np.uint8), 1-alpha, 0)  # 컬러 값 0(검은색)

In [ ]:
draw_img(cv2.cvtColor(back_img_show, cv2.COLOR_BGR2RGB), 'back_n_cat')

- 이미지 처리 결과 저장

In [ ]:
# cf. plt.savefig() 는 plt.show() 직전에 해야 처리가 끝난 이미지 저장이 제대로 된다.
# plt.savefig('image.png', bbox_inches='tight',pad_inches = 0) 로는 공백 및 테두리 숨기기가 제대로 안 됐기에 plt.imsave('image.png', img)를 사용함
plt.imsave("result_imgs/back_n_cat.png", cv2.cvtColor(back_img_show, cv2.COLOR_BGR2RGB))

- 합치기 전후 비교

In [ ]:
draw_imgs(cv2.cvtColor(BACK_IMG_ORIG, cv2.COLOR_BGR2RGB), cv2.cvtColor(back_img_show, cv2.COLOR_BGR2RGB), 'BACK_IMG_ORIG', 'back_n_cat')

### Step 2. 사진에서 문제점 찾기

이미지의 경계선 처리가 깔끔하지 못하다.

![](result_imgs/person_blur.png)

![](result_imgs/cat_blur.png)

### Step 3. 해결 방법을 제안해 보기

인물모드 사진의 문제점을 해결할 방안 제시하기
- 단순히 'XXX 기술을 사용한다.' 정도의 선언적 솔루션 X
- => 내가 선택한 기술이 DeepLab 모델의 Semantic Segmentation 이 만들어 낸 Mask 영역에 어떻게 적용되어 문제점을 보완하게 되는지의 메커니즘이 포함된 솔루션 O

#### My Solution

경계선이 자연스러워 보일 수 있는 방법 생각/검색/시도한 목록

1. 엣지 검출 전 샤프닝 적용
2. 엣지 검출하기 위해 캐니엣지 적용: cv2.Canny(img, 임계값, 임계값)
3. 블러링 함수로 cv2.blur() 대신 다른 함수 사용해보기
4. 배경 제거 함수로 cv2.bgsegm.createBackgroundSubtractorMOG(history, nmixtures, backgroundRatio, noiseSigma) 사용해보기

=> 전부 실패했다. 

5. 아예 다른 모델 사용하기
- MODNet

In [ ]:
import os
from git import Repo

# clone the repository
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet
%cd MODNet/

# dowload the pre-trained ckpt for image matting
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt

In [ ]:
import shutil
import os
from google.colab import files

# clean and rebuild the image folders
input_folder = 'demo/image_matting/colab/input'
if os.path.exists(input_folder):
  shutil.rmtree(input_folder)
os.makedirs(input_folder)

output_folder = 'demo/image_matting/colab/output'
if os.path.exists(output_folder):
  shutil.rmtree(output_folder)
os.makedirs(output_folder)

# upload images (PNG or JPG)
image_names = list(files.upload().keys())
for image_name in image_names:
  shutil.move(image_name, os.path.join(input_folder, image_name))


In [ ]:
import shutil
import os
from google.colab import files

# clean and rebuild the image folders
input_folder = 'demo/image_matting/colab/input'
if os.path.exists(input_folder):
  shutil.rmtree(input_folder)
os.makedirs(input_folder)

output_folder = 'demo/image_matting/colab/output'
if os.path.exists(output_folder):
  shutil.rmtree(output_folder)
os.makedirs(output_folder)

# upload images (PNG or JPG)
image_names = list(files.upload().keys())
for image_name in image_names:
  shutil.move(image_name, os.path.join(input_folder, image_name))


In [ ]:
!python -m demo.image_matting.colab.inference \
        --input-path demo/image_matting/colab/input \
        --output-path demo/image_matting/colab/output \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt

In [ ]:
import numpy as np
from PIL import Image

def combined_display(image, matte):
  # calculate display resolution
  w, h = image.width, image.height
  rw, rh = 800, int(h * 800 / (3 * w))
  
  # obtain predicted foreground
  image = np.asarray(image)
  if len(image.shape) == 2:
    image = image[:, :, None]
  if image.shape[2] == 1:
    image = np.repeat(image, 3, axis=2)
  elif image.shape[2] == 4:
    image = image[:, :, 0:3]
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  
  # combine image, foreground, and alpha into one line
  combined = np.concatenate((image, foreground, matte * 255), axis=1)
  combined = Image.fromarray(np.uint8(combined)).resize((rw, rh))
  return combined

# visualize all images
image_names = os.listdir(input_folder)
for image_name in image_names:
  matte_name = image_name.split('.')[0] + '.png'
  image = Image.open(os.path.join(input_folder, image_name))
  matte = Image.open(os.path.join(output_folder, matte_name))
  display(combined_display(image, matte))
  print(image_name, '\n')

In [ ]:
zip_filename = 'matte.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)

os.system(f"zip -r -j {zip_filename} {output_folder}/*")
files.download(zip_filename)

- 엣지 검출 전 샤프닝 적용

In [ ]:
# 언샤프 마스크 필터링 https://deep-learning-study.tistory.com/155

img = cv2.imread(PERSON_IMG_PATH, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(img, (0, 0), 2) # 표준편차 2, 필터 크기는 자동 설정

a = 2.0 # 샤프닝 정도 결정하는 변수
dst = np.clip((1+a)*img - a * blr, 0, 255).astype(np.uint8) # 계산은 정수, 출력은 실수

draw_imgs(cv2.cvtColor(PERSON_IMG_ORIG, cv2.COLOR_BGR2RGB), cv2.cvtColor(dst, cv2.COLOR_BGR2RGB), 'before', 'after')

plt.imsave('result_imgs/person_sharpening.png', cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))

In [ ]:
# 이미지 대조 색상 강조하기 https://ansan-survivor.tistory.com/311

''' 함수) 이미지를 더 선명하게 Contrast(대조) 기법을 적용시킴.
param : 컬러 이미지
return : 대조된 이미지
'''

def img_Contrast(img):
    # -----Converting image to LAB Color model-----------------------------------
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    # -----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)
    
    # -----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    
    # -----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl, a, b))
    
    # -----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

img = cv2.imread('result_imgs/person_sharpening.png', cv2.COLOR_BGR2GRAY)
img = img_Contrast(img)

plt.imsave("result_imgs/person_contrast.png", cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

- 보행자 검출을 위한 HOG
    - 이미지가 아니라 영상에서 사람을 검출하기 위한 용도의 코드지만 혹시나 사용해봤다. 역시 제대로 작동하지 않았다.

In [ ]:
import random
import sys

# 동영상 불러오기
# img = cv2.imread('human_segmentation/images/couple.jpg', cv2.COLOR_BGR2RGB)
img = dst.copy()

# 보행자 검출을 위한 HOG 기술자 설정
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# 매 프레임마다 보행자 검출
detected, _ = hog.detectMultiScale(img) # 사각형 정보를 받아옴
print(detected)

# 검출 결과 화면 표시
for (x, y, w, h) in detected:
    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.rectangle(img, (x, y, w, h), c, 3)

####
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

- 엣지 검출 좀 더 정확히 해보기

In [ ]:
# 캐니 엣지 적용
edged = cv2.Canny(img, 300, 600)  # 하단 임계값과 상단 임계값은 실험적으로 결정하기

draw_img(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

plt.imsave("result_imgs/person_canny_edge.png", cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

- 블러링 함수로 cv2.blur() 대신 다른 함수 사용해보기
    - cv2.GaussianBlur()
    - cv2.MedianBlur()

In [ ]:
# 모델에 이미지 입력
dst_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)

edged_path = 'result_imgs/person_edged.png'
person_2_segvalues, person_2_output = MODEL.segmentAsPascalvoc(edged_path)
for class_id in person_2_segvalues['class_ids']:
    print(LABEL_NAMES[class_id])
draw_img(person_2_output, 'person_2_output')

seg_color = get_colormap('person')
person_2_seg_map = np.all(person_2_output==seg_color, axis=-1)

person_2_img_mask = person_2_seg_map.astype(np.uint8) * 255

# person_2_img_orig_blur = cv2.medianBlur(dst_gray, 21)
person_2_img_orig_blur = cv2.GaussianBlur(dst_gray, (13,13), 0)

# person_2_img_mask_color = cv2.cvtColor(person_2_img_mask, cv2.COLOR_GRAY2BGR)
person_2_img_mask_color = person_2_img_mask.copy()
person_2_img_bg_mask = cv2.bitwise_not(person_2_img_mask)
person_2_img_bg_blur = cv2.bitwise_and(person_2_img_orig_blur, person_2_img_bg_mask)

draw_img(cv2.cvtColor(person_2_img_bg_blur, cv2.COLOR_BGR2RGB))

person_2_img_concat = np.where(person_2_img_mask_color==255, dst_gray, person_2_img_bg_blur)

draw_img(cv2.cvtColor(person_2_img_concat, cv2.COLOR_BGR2RGB))

plt.imsave("result_imgs/person_gaussian_blur.png", cv2.cvtColor(person_2_img_concat, cv2.COLOR_BGR2RGB))

In [ ]:
image = img.copy()
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
blur = cv2.GaussianBlur(image_gray, ksize=(3,3), sigmaX=0)
ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)

In [ ]:
blur = cv2.GaussianBlur(image_gray, ksize=(5,5), sigmaX=0)
ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)

In [ ]:
edged = cv2.Canny(blur, 10, 250)
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

plt.imsave('result_imgs/person_edged.png', cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
plt.imshow(cv2.cvtColor(closed, cv2.COLOR_BGR2RGB))

In [ ]:
contours, _ = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
total = 0

In [ ]:
# 외곽선 그리는 용도. 이미지에 그리기 때문에 이 코드 적용하면 원래 이미지에
# 초록색 선 생김
contours_image = cv2.drawContours(image, contours, -1, (0,255,0), 3)
plt.imshow(cv2.cvtColor(contours_image, cv2.COLOR_BGR2RGB))

- 배경 제거 함수로 cv2.bgsegm.createBackgroundSubtractorMOG(history, nmixtures, backgroundRatio, noiseSigma) 사용해보기
    - 작동하지 않음!
    - 원래 이미지가 아니라 영상에서 사용하던 함수여서 그런 듯하다.

In [ ]:
fgbg = cv2.createBackgroundSubtractorMOG2()

fgmask = fgbg.apply(img)

plt.figure(figsize=(12,12))

plt.subplot(211)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.subplot(212)
plt.imshow(cv2.cvtColor(fgmask, cv2.COLOR_BGR2RGB))

plt.show()

---

# 회고

### - 이번 프로젝트에서 **어려웠던 점**.
저작권 없는 이미지 사이트에서 사진 고르느라 시간 다 뺏겼다. 경계선을 자연스럽게 만드는 방법을 못 찾겠다.

### - 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
맨 아래에 내용 정리를 해놓았음

### - 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.

>#### **루브릭**
>|평가문항|상세기준|
>|:---:|:---:|
>|1. 인물모드 사진을 성공적으로 제작하였다.|아웃포커싱 효과가 적용된 본인의 인물모드 사진과 고양이 사진, 배경전환 크로마키사진을 각각 1장 이상 성공적으로 제작하였다.|
>|2. 제작한 인물모드 사진들에서 나타나는 문제점을 정확히 지적하였다.|인물사진에서 발생한 문제점을 정확히 지적한 사진을 제출하였다.|
>|3. 인물모드 사진의 문제점을 개선할 수 있는 솔루션을 적절히 제시하였다.|추가적인 depth 정보를 활용하여 semantic segmentation mask의 오류를 보완할 수 있는 좋은 솔루션을 구체적으로 제시하였다.|

사람 사진, 고양이 사진으로 아웃포커싱 효과를 적용했다. 배경 이미지 속 커튼에 고양이가 매달려있는 귀여운 모습을 완성시켰다.

다른 문제점은 잘 모르겠고 경계선이 부자연스러운 것만 해결하면 괜찮다고 생각했다.

원본 이미지의 경계선을 샤프닝을 통해 더 뚜렷하게 만들면 어떨까 시도해봤지만 DeepLab 모델에 넣어봤더니 딱히 달라진 것은 없었다. 블러링 함수를 바꿔봤지만 여전히 달라진 것은 없었다. 배경 제거 함수를 사용해 봤지만 동작하지 않았다. 짜잘한 건 효과가 없는 모양이다. 그래서 모델을 바꾸는 것이 좋다고 결론내렸다. 찾아보니 MODNet 이라는 모델이 DeepLab처럼 사람 검출을 잘하는 것으로 나와서 이걸 사용하면 좋겠다고 적었다.


### - 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.
없음

### - **자기 다짐**
이제 안 되는 것은 타협하고 아이펠 선배의 우수 노드를 참고해서 넘어가야겠다.

---
- 대나무숲

라이브러리 때문에 환장하겠다. 이놈의 아나콘다 또 꼬여서 빨간 에러만 잔뜩 뜨길래 revision 했더니 vs code에서 python 실행이 아예 막혀버렸다. 정확히 말하자면 실행이 먹히지가 않고 그냥 무응답이다. vs code의 python interpreter 설정 리셋을 눌러도 반응이 없다. 너무 답답해서 터미널로 conda info 쳐봤더니 conda env가 activate가 안 됐다고 나온다. 내가 몇 번을 activate 했는데도 저러길래 vs code 삭제 & 재설치까지 했다. 그래도 안 된다. 구글에 검색해봐도 나와 같은 현상을 겪는 사람은 아무도 없다. vs code에 설정된 python interpreter가 vs code 삭제했는데도 그대로 남아있어서 이걸 제거해야 하는데 어디에 있는지 찾을 수가 없다. 해결 방법이 없다... 결국 env 삭제하고 새로 만들어서 패키지 설치하는 중인데 또 다시 새빨간 에러로 도배됐다. 주말동안 이거 때문에 주피터 노트북 삭제 & 재설치를 몇 번을 했는데 또또또 원점이다. 다시 처음부터 주피터 노트북 삭제하고 설치해야 한다. 쓸데없는 걸로 시간 다 날리고 스트레스가 너무 심하다. 왜 그렇게 화나는지 빨간 색 에러의 일부를 보여주겠다. '일부'라서 저것보다 더 많다는 것을 염두해 둬야 할 것이다.

![](result_imgs/error.png)

---

### 내용 정리

- Image Blurring
  - Image Blurring == Image Smoothing
  - 용도: 노이즈 제거, 경계선 흐리게 만들기
  - OpenCV에는 4가지 blurring 방법 제공
    - Averaging
      - Box형태의 kernel을 이미지에 적용한 후 평균값을 box의 중심점에 적용하는 형태
      - 커널
        - box filter는 동일한 값으로 구성된 kernel 사용
        - e.g. 3x3 box kernel: $K = \frac{1}{9} \begin{bmatrix}1&1&1\\1&1&1\\1&1&1 \end{bmatrix}$
      - 함수
        - cv2.blur(src, ksize) → dst: 정규화된 box filter로 이미지 컨볼루젼
        - cv2.boxFilter(src, ksize): 정규화되지 않은 box filter로 이미지 컨볼루젼
    - Gaussian Filtering
      - kernel 행렬의 값을 Gaussian 함수를 통해서 수학적으로 생성하여 적용
      - kernel의 사이즈 = 양수 + 홀수로 지정
      - 이미지의 [Gaussian Noise](https://en.wikipedia.org/wiki/Gaussian_noise) (전체적으로 밀도가 동일한 노이즈, 백색노이즈)를 제거하는 데 가장 효과적
      - cv2.GaussianBlur(img, ksize, sigmaX)
    - Median Filtering
      - kernel window와 pixel의 값들을 정렬한 후에 중간값을 선택하여 적용
      - [salt-and-pepper noise](https://ko.wikipedia.org/wiki/%EC%A0%90%EC%9E%A1%EC%9D%8C) 제거에 가장 효과적
      - cv2.medianBlur(src, ksize)
    - Bilateral Filtering
      - Bilateral Filtering(양방향 필터)은 경계선을 유지(경계선 흐려짐X)하면서 Gaussian Blur처리
      - cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)